In [15]:

from langchain_core.messages import SystemMessage, HumanMessage

import sys
import os
from dotenv import load_dotenv
load_dotenv()


# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [16]:
from langchain_groq import ChatGroq


llm = ChatGroq(model="gemma2-9b-it")

emailai = llm


In [17]:
email_instruction = """
Prompt:

"You are an AI communication specialist focused on client retention.
Based on the provided client data and risk score, craft a personalized email with the following instructions:

Subject Line: Friendly and relevant to the client’s situation.

Body: Acknowledge the client’s relationship, express understanding of their needs, and proactively offer continued support or next steps.

Tone Adjustment:

If risk score is low, maintain a positive, appreciative tone.

If risk score is medium, be reassuring and offer incentives or check-ins.

If risk score is high, be highly supportive, express urgency in resolving concerns, and highlight benefits of staying connected.

Input:
{client_data}

Output:
Return only the subject and body of the email in markdown format, clearly separated."
"""


In [18]:
from typing import Any, Dict
from typing_extensions import TypedDict


# Graph state
class State(TypedDict):
    input_data: Dict[str, Any]  # user query
    response: str  # response


In [19]:
def email_ai(state: State):
    """AI agent that understand user query and generate string for news extraction"""
    try:
        response = emailai.invoke(
            [
            SystemMessage(content=email_instruction),
            HumanMessage(
                content=f"Here is the user query: {state['input_data']}"
            ),
        ]
    )

        return {"response": response}
    except Exception as e:
        raise Exception(e, sys)

In [20]:
from langgraph.graph import StateGraph, START, END

def email_generator():
    # Build workflow
    builder = StateGraph(State)

    # Add the nodes
    builder.add_node("email_ai", email_ai)

    # Add edges to connect nodes
    builder.add_edge(START, "email_ai")
    builder.add_edge("email_ai", END)

    # Compile the workflow
    graph = builder.compile()

    return graph

email = email_generator()

In [21]:
data = {"Client Name": "john",
"Policy Type": "car",
"Policy Start Date": "2020-01-01",
"Annual Premium": 1000,
"Claims in Last 3 Years": 0,
"Communication Frequency": "monthly",
"Client Tenure (Years)": 3,
"Industry": "finance",
"Region": "New York",
"Risk Score": 1
}

In [22]:
result = email.invoke({"input_data": data})

In [23]:
result["response"].content

"Subject: Keeping Your Ride Protected, John!\n\nBody: \n\nHi John,\n\nWe wanted to reach out and say thank you for being a valued customer for the past 3 years! We appreciate your continued trust in us for your car insurance needs.\n\nAs your policy anniversary approaches, we want to make sure everything is still running smoothly.  Do you have any questions about your coverage or would you like to discuss any adjustments to your policy? We're here to help!\n\n\n\n"